In [120]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import cmath
import math
import time

In [121]:
# Fixed Parameters
S0 = 1900
K = 2100
k = math.log(K)
r = 0.02
q = 0.0187

In [122]:
# Parameters for FFT and FrFFT 

n_FFT = 9
N_FFT = 2**n_FFT

n_FrFFT = 9
N_FrFFT = 2**n_FrFFT

N = 2**9

#step-size
eta = 0.25
# damping factor
alpha = 0.4


In [123]:
# step-size in log strike space
lda_FFT = (2*math.pi/N_FFT)/eta # lda is fixed under FFT
lda_FrFFT = 0.001 # lda is an adjustable parameter under FrFFT, 


#Choice of beta
beta = np.log(S0)-N*lda_FFT/2
#beta = np.log(S0)-N*lda_FrFFT/2
#beta = np.log(K)

In [124]:
#model-specific Parameters
model = 'BlackScholes'

In [125]:
params = []     
if (model == 'GBM'):
    
    sig = 0.30
    params.append(sig);
    
elif (model == 'VG'):
    
    sig = 0.3
    nu = 0.5
    theta = -0.4
    #
    params.append(sig);
    params.append(nu);
    params.append(theta);

elif model == 'BlackScholes':  
    sigma = 0.36  # Volatility  
    params.append(sigma)  
    
elif (model == 'Heston'):
    
    kappa = 2.0
    theta = 0.05
    sig = 0.30
    rho = -0.70
    v0 = 0.04
    #
    params.append(kappa)
    params.append(theta)
    params.append(sig)
    params.append(rho)
    params.append(v0)
    

In [126]:
def generic_CF(u, params, S0, r, q, T, model):
    
    if (model == 'GBM'):
        
        sig = params[0]
        mu = np.log(S0) + (r-q-sig**2/2)*T
        a = sig*np.sqrt(T)
        phi = np.exp(1j*mu*u-(a*u)**2/2)

    elif (model == 'BlackScholes'):
        sigma = params[0]  # Volatility

        mu = np.log(S0) + (r - q - 0.5 * sigma**2) * T  # Drift
        a = sigma * np.sqrt(T)  # Standard deviation over the maturity period
        phi = np.exp(1j * mu * u - 0.5 * a**2 * u**2)  # Characteristic function for Black-Scholes
        
    elif(model == 'Heston'):
        
        kappa  = params[0]
        theta  = params[1]
        sigma  = params[2]
        rho    = params[3]
        v0     = params[4]
        
        tmp = (kappa-1j*rho*sigma*u)
        g = np.sqrt((sigma**2)*(u**2+1j*u)+tmp**2)
        
        pow1 = 2*kappa*theta/(sigma**2)
        
        numer1 = (kappa*theta*T*tmp)/(sigma**2) + 1j*u*T*r + 1j*u*math.log(S0)
        log_denum1 = pow1 * np.log(np.cosh(g*T/2)+(tmp/g)*np.sinh(g*T/2))
        tmp2 = ((u*u+1j*u)*v0)/(g/np.tanh(g*T/2)+tmp)
        log_phi = numer1 - log_denum1 - tmp2
        phi = np.exp(log_phi)
        
        #g = np.sqrt((kappa-1j*rho*sigma*u)**2+(u*u+1j*u)*sigma*sigma)
        #beta = kappa-rho*sigma*1j*u
        #tmp = g*T/2
        
        #temp1 = 1j*(np.log(S0)+(r-q)*T)*u + kappa*theta*T*beta/(sigma*sigma)
        #temp2 = -(u*u+1j*u)*v0/(g/np.tanh(tmp)+beta)
        #temp3 = (2*kappa*theta/(sigma*sigma))*np.log(np.cosh(tmp)+(beta/g)*np.sinh(tmp))
        
        #phi = np.exp(temp1+temp2-temp3);
        

    elif (model == 'VG'):
        
        sigma  = params[0];
        nu     = params[1];
        theta  = params[2];

        if (nu == 0):
            mu = np.log(S0) + (r-q - theta -0.5*sigma**2)*T
            phi  = np.exp(1j*u*mu) * np.exp((1j*theta*u-0.5*sigma**2*u**2)*T)
        else:
            mu  = np.log(S0) + (r-q + np.log(1-theta*nu-0.5*sigma**2*nu)/nu)*T
            phi = np.exp(1j*u*mu)*((1-1j*nu*theta*u+0.5*nu*sigma**2*u**2)**(-T/nu))

    return phi

In [127]:
def evaluateIntegral(params, S0, K, r, q, T, alpha, eta, N, model):
    
    # Just one strike at a time
    # no need for Fast Fourier Transform
    
    # discount factor
    df = math.exp(-r*T)
    
    sum1 = 0
    for j in range(N):
        nuJ = j*eta
        psi_nuJ = df*generic_CF(nuJ-(alpha+1)*1j, params, S0, r, q, T, model)/((alpha + 1j*nuJ)*(alpha+1+1j*nuJ))
        if j == 0:
            wJ = (eta/2)
        else:
            wJ = eta
        sum1 += np.exp(-1j*nuJ*k)*psi_nuJ*wJ
        
    cT_k = (np.exp(-alpha*k)/math.pi)*sum1
    
    return np.real(cT_k) 

In [128]:
def genericFFT(params, S0, K, r, q, T, alpha, eta, n, model):
    
    N = 2**n
    
    # step-size in log strike space
    lda = (2*np.pi/N)/eta
    
    #Choice of beta
    #beta = np.log(S0)-N*lda/2
    #beta = np.log(K)
    
    # forming vector x and strikes km for m=1,...,N
    km = np.zeros((N))
    xX = np.zeros((N))
    
    # discount factor
    df = math.exp(-r*T)
    
    nuJ = np.arange(N)*eta
    psi_nuJ = generic_CF(nuJ-(alpha+1)*1j, params, S0, r, q, T, model)/((alpha + 1j*nuJ)*(alpha+1+1j*nuJ))
    
    for j in range(N):  
        km[j] = beta+j*lda
        if j == 0:
            wJ = (eta/2)
        else:
            wJ = eta
            
        xX[j] = np.exp(-1j*beta*nuJ[j])*df*psi_nuJ[j]*wJ
     
    yY = np.fft.fft(xX)
    cT_km = np.zeros((N))  
    for i in range(N):
        multiplier = np.exp(-alpha*km[i])/math.pi
        cT_km[i] = multiplier*np.real(yY[i])
    
    return km, cT_km

In [129]:
def genericFrFFT(params, S0, K, r, q, T, alpha, eta, n, lda, model):
    
    N = 2**n
    gamma = eta*lda/(2*math.pi)

    #Choice of beta
    #beta = np.log(S0)-N*lda/2
    beta = np.log(K)

    # initialize x, y, z, and cT_km
    km = np.zeros((N))
    x = np.zeros((N))
    y = np.zeros((2*N), dtype=np.complex128)
    z = np.zeros((2*N), dtype=np.complex128)
    cT_km = np.zeros((N)) 

    # discount factor
    df = math.exp(-r*T)

    # compute x
    nuJ = np.arange(N)*eta
    psi_nuJ = generic_CF(nuJ-(alpha+1)*1j, params, S0, r, q, T, model)/((alpha + 1j*nuJ)*(alpha+1+1j*nuJ))

    for j in range(N):  
        km[j] = beta+j*lda
        if j == 0:
            wJ = (eta/2)
        else:
            wJ = eta
        x[j] = np.exp(-1j*beta*nuJ[j])*df*psi_nuJ[j]*wJ

    # set up y
    for i in range(N):
        y[i] = np.exp(-1j*math.pi*gamma*i**2)*x[i]
    y[N:] = 0

    # set up z
    for i in range(N):
        z[i] = np.exp(1j*math.pi*gamma*i**2)
    z[N:] = z[:N][::-1]

    # compute xi_hat
    xi_hat = np.fft.ifft(np.fft.fft(y) * np.fft.fft(z))

    # compute call prices
    for i in range(N):
        cT_km[i] = np.exp(-alpha*(beta + i*lda))/math.pi * (np.exp(-1j*math.pi*gamma*i**2)*xi_hat[i]).real

    return km, cT_km

In [130]:
print(' ')
print('===================')
print('Model is %s' % model)
print('-------------------')
    
T = 0.25

# FFT
print(' ')
start_time = time.time()
km, cT_km = genericFFT(params, S0, K, r, q, T, alpha, eta, n_FFT, model)
#cT_k = cT_km[0]
cT_k = np.interp(k, km, cT_km)

elapsed_time = time.time() - start_time
    
#cT_k = np.interp(np.log(k), km, cT_km)
print("Option via FFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_k))
#print("Option via FFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_km[0]))
print('FFT execution time was %0.7f' % elapsed_time)

# FrFFT
print(' ')
start_time = time.time()
km, cT_km = genericFrFFT(params, S0, K, r, q, T, alpha, eta, n_FrFFT, lda_FrFFT, model)
#cT_k = cT_km[0]
cT_k = np.interp(k, km, cT_km)

elapsed_time = time.time() - start_time
    
#cT_k = np.interp(np.log(), km, cT_km)
print("Option via FrFFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_k))
#print("Option via FFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_km[0]))
print('FrFFT execution time was %0.7f' % elapsed_time)


# Integral
print(' ')
start_time = time.time()
cT_k = evaluateIntegral(params, S0, K, r, q, T, alpha, eta, N, model)
elapsed_time = time.time() - start_time
print("Option via Integration: for strike %s the option premium is %6.4f" % (np.exp(k), cT_k))
print('Evaluation of integral time was %0.7f' % elapsed_time)

 
Model is BlackScholes
-------------------
 
Option via FFT: for strike 2100.0 the option premium is 142.8699
FFT execution time was 0.0158188
 
Option via FrFFT: for strike 2100.0 the option premium is 64.9174
FrFFT execution time was 0.0230877
 
Option via Integration: for strike 2100.0 the option premium is 64.9160
Evaluation of integral time was 0.0160797


In [1]:
## Title:        Option Pricing
## Author:       Elisa FLeissner, Lars Stauffenegger
## Email:        elisa.fleissner@student.unisg.ch,
##               lars.stauffenegger@student.unisg.ch,
## Place, Time:  Zürich, 24.03.19
## Description:  Option Pricing with Black Scholes, COS Method and Heston Model
## Improvements: -
## Last changes: -


# In[1]: Packages
import quandl
import numpy as np
import AllFunctions as func
import matplotlib.pyplot as plt


# In[2]: Data
# Import from quandl
quandl.ApiConfig.api_key = "mrMTRoAdPycJSyzyjxPN"
ticker     = "AAPL"
database   = "EOD"
identifier = database + "/" + ticker
stockData  = quandl.get(identifier, rows = 500)

# Return and Volatility
logReturn = np.log(stockData.Close) - np.log(stockData.Close.shift(1))
logReturn.drop(logReturn.index[:1], inplace = True)
tradingDaysCount   = 252
annualisedMean     = np.mean(logReturn) * tradingDaysCount
annualisedVariance = np.var(logReturn) * tradingDaysCount
annualisedStdDev   = np.sqrt(annualisedVariance)
lastPrice          = stockData.Close.tail(1)


# In[2]: Parameter
# Volvol and rho according to Fang, 2010, p. 30
r      = 0                  # assumption Risk-free rate
mu     = r #annualisedMean  # Mean rate of drift
sigma  = annualisedStdDev   # Initial Vola of underyling at time 0; also called u0 or a
S0     = lastPrice[0]       # Today's stock price
tau    = 30 / 365           # Time to expiry in years
q      = 0                  # Divindend Yield
lm     = 1.5768             # The speed of mean reversion
v_bar  = annualisedVariance # Mean level of variance of the underlying
volvol =  0.5751            # Volatility of the volatiltiy process
rho    = -0.5711            # Covariance between the log stock and the variance process

# Range of Strikes
mini    = int(S0 * 0.8)
maxi    = int(S0 * 1.2)
K       = np.arange(mini, maxi, dtype = np.float)

# Truncation Range
L       = 120
a, b    = func.truncationRange(L, mu, tau, sigma, v_bar, lm, rho, volvol)
bma     = b-a

# Number of Points
N       = 15
k       = np.arange(np.power(2,N))

# Input for the Characterstic Function Phi
u       = k * np.pi/bma


# In[3]: Black Scholes Option Pricing
C_BS, P_BS = func.blackScholes(S0, K, r, tau, sigma, q)
print(C_BS)


# In[4]: COS-FFT Value Function for Put
UkPut  = 2 / bma * ( func.cosSer1(a,b,a,0,k) - func.cosSerExp(a,b,a,0,k) )
UkCall = 2 / bma * ( func.cosSerExp(a,b,0,b,k) - func.cosSer1(a,b,0,b,k) )


# In[5]: COS with BS-Characterstic Function
charactersticFunctionBS = func.charFuncBSM(u, mu, sigma, tau)

C_COS = np.zeros((np.size(K)))

for m in range(0,np.size(K)):
    x  = np.log(S0/K[m])
    addIntegratedTerm = np.exp(1j * k * np.pi * (x-a)/bma)
    Fk = np.real(np.multiply(charactersticFunctionBS, addIntegratedTerm))
    Fk[0]=0.5 * Fk[0] 
    C_COS[m] = K[m] * np.sum(np.multiply(Fk,UkCall)) * np.exp(-r * tau)
    
print (C_COS)


# In[6]: COS with Fang & Oosterlee (2008) Version of Heston's Characteristic Function
charactersticFunctionHFO = func.charFuncHestonFO(mu, r, u, tau, sigma, v_bar, lm, rho, volvol)

C_COS_HFO = np.zeros((np.size(K)))
P_COS_HFO = np.zeros((np.size(K)))
C_COS_PCP = np.zeros((np.size(K)))

for m in range(0, np.size(K)):
    x  = np.log(S0/K[m])
    addIntegratedTerm = np.exp(1j * k * np.pi * (x-a)/bma)
    Fk = np.real(charactersticFunctionHFO * addIntegratedTerm)
    Fk[0] = 0.5 * Fk[0]						
    C_COS_HFO[m] = K[m] * np.sum(np.multiply(Fk, UkCall)) * np.exp(-r * tau)
    P_COS_HFO[m] = K[m] * np.sum(np.multiply(Fk, UkPut)) * np.exp(-r * tau)
    C_COS_PCP[m] = P_COS_HFO[m] + S0 * np.exp(-q * tau) - K[m] * np.exp(-r * tau)

print(C_COS_HFO)
print(P_COS_HFO)
print(C_COS_PCP)


# In[7]: Plotting
plt.plot(K, C_BS, "g.", K, C_COS, "b.", K, C_COS_HFO, "r.")
plt.axvline(x = S0)
plt.show()
print("C_BS = green, C_COS = blue, C_COS_HFO = red")

## End

ModuleNotFoundError: No module named 'AllFunctions'

In [2]:
# !pip install quandl

! pip install AllFunctions

ERROR: Could not find a version that satisfies the requirement AllFunctions (from versions: none)
ERROR: No matching distribution found for AllFunctions


In [2]:
pip show quandl


Name: Quandl
Version: 3.7.0
Summary: Package for quandl API access
Home-page: https://github.com/quandl/quandl-python
Author: Quandl
Author-email: connect@quandl.com
License: MIT
Location: /Users/ego/opt/anaconda3/lib/python3.9/site-packages
Requires: inflection, more-itertools, numpy, pandas, python-dateutil, requests, six
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install quandl --upgrade --no-cache-dir


Note: you may need to restart the kernel to use updated packages.
